In [1]:
from gtts import gTTS
import speech_recognition as sr
#from subprocess import call     # MAC / LINUX
from playsound import playsound # WINDOWS
from requests import get
from bs4 import BeautifulSoup
import webbrowser as browser
import json
import os
import urllib.request    # ... para pagina que no funcionan con RSS o FEED ...
from datetime import date
from datetime import datetime

#Día actual
today = date.today()
dia_actual = format(today.day)
mes_actual = format(today.month)
mes_actual = int(mes_actual)
anho_actual = format(today.year)
meses = ("Enero", "Febrero", "Marzo", "Abri", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre")
mesliteral = meses[mes_actual - 1]


import time
#Hora actual
hora_actual = time.strftime("%H:%M:%S") #Formato de 24 horas


##### CONFIGURACIONES #####
#hotword = 'natalia'
hotword = 'atenea'

#with open('camilapythonasistente-7723e4c012dd.json') as credenciais_google:
with open('asistentepython2-2b41fbcac156.json') as credenciales_google:
    credenciales_google = credenciales_google.read()  
    
    
##### FUNCIONES PRINCIPALES #####

def monitora_audio():
    microfone = sr.Recognizer()
    with sr.Microphone() as source:
        while True:
            print("Esperando el  comando: ")
            audio = microfone.listen(source)
            try:
                trigger = microfone.recognize_google_cloud(audio, credentials_json=credenciales_google, language='es-us')
                trigger = trigger.lower()

                if hotword in trigger:
                    print('COMANDO: ', trigger)
                    responde('feedback_es')
                    ejecutar_comandos(trigger)
                    break

            except sr.UnknownValueError:
                print("Google no entiende ese audio")
            except sr.RequestError as e:
                print("Could not request results from Google Cloud Speech service; {0}".format(e))
    return trigger

def responde(arquivo):
    #call(['afplay', 'audios/' + arquivo + '.mp3']) # OSC
    playsound('audios/' + arquivo + '.mp3')        # WINDOWS

def crear_audio(mensagem):
    os.remove('audios/mensagem.mp3')
    tts = gTTS(mensagem, lang='es-us')
    tts.save('audios/mensagem.mp3')
    print('ATHENEA: ', mensagem)
    playsound('audios/mensagem.mp3')        # WINDOWS
    #call(['afplay', 'audios/mensagem.mp3']) # OSX
    


def ejecutar_comandos(trigger):
    if 'noticias' in trigger:
        ultimas_noticias()
        
    elif 'locales' in trigger:
        noticias_locales()
        
    elif 'deportes' in trigger:
        noticias_deportivas()

    elif 'tocar' in trigger and 'bee gees' in trigger:
        playlists('bee_gees')

    elif 'tocar' in trigger and 'taylor davis' in trigger:
        playlists('taylor_davis')
        
    elif 'tocar' in trigger and 'trio los angeles' in trigger:
        playlists('trio_los_angeles')
        
    elif 'tocar' in trigger and 'mambole' in trigger:
        playlists('mambole')
        
    elif 'tocar' in trigger and 'josé augusto' in trigger:
        playlists('jose_augusto')     

    elif 'temperatura actual' in trigger:
        prevision_tiempo(tiempo=True)

    elif 'estado del tiempo' in trigger:
        prevision_tiempo(minmax=True)

    else:
        mensagem = trigger.strip(hotword)
        crear_audio(mensagem)
        print('COMANDO INVALIDO', mensagem)
        responde('comando_invalido_es')


##### FUNCIONES COMANDOS #####

def ultimas_noticias():
    mensaje = 'Soy Atenea periodista virtual y éstas son las noticias nacionales del momento en radio estacion.cl, para el día de hoy '+dia_actual+' de '+ mesliteral+' de '+ anho_actual+', siendo las '+ hora_actual
    crear_audio(mensaje)
    
    site = get('https://news.google.com/news/rss?ned=es_cl&gl=CL&hl=cl')
    noticias = BeautifulSoup(site.text, 'html.parser')
    for item in noticias.findAll('item')[:5]:
        mensagem = item.title.text
        crear_audio(mensagem)
        
def noticias_locales():
    site = get('https://aricaonline.cl/feed/')
    noticias = BeautifulSoup(site.text, 'html.parser')
    for item in noticias.findAll('item')[:5]:
        mensagem = item.title.text
        crear_audio(mensagem +' ... - aricaonline.cl')  
        
    site = get('https://aricaldia.cl/feed/')
    noticias = BeautifulSoup(site.text, 'html.parser')
    for item in noticias.findAll('item')[:5]:
        mensagem = item.title.text
        crear_audio(mensagem +' - aricaldia.cl')  
        
    site = get('https://aricahoy.cl/feed/')
    noticias = BeautifulSoup(site.text, 'html.parser')
    for item in noticias.findAll('item')[:5]:
        mensagem = item.title.text
        crear_audio(mensagem +' ... - aricahoy.cl')         
        

def noticias_deportivas():
    datos = urllib.request.urlopen('https://news.google.com/topics/CAAqLAgKIiZDQkFTRmdvSUwyMHZNRFp1ZEdvU0JtVnpMVFF4T1JvQ1ZWTW9BQVAB?hl=es-419&gl=CL&ceid=CL:es-419').read().decode()
    datos
    soup = BeautifulSoup(datos)
    tags=soup('a')
       
    numero_enlace=0   # para saltar los enlaces irrelevantes ...
    
    for tag in tags:
        numero_enlace +=1
        if numero_enlace >= 52:
            mensagem = tag.text
            if mensagem != '' and mensagem !='Ver cobertura completa':
                crear_audio(mensagem)
 
        
def playlists(album):
    if album == 'bee_gees':
        browser.open('https://open.spotify.com/track/33ALuUDfftTs2NEszyvJRm')
    elif album == 'taylor_davis':
        browser.open('https://open.spotify.com/track/3MKep4BfEwSlAHuFJrA9aV')
    elif album == 'trio_los_angeles':
        browser.open('https://www.youtube.com/watch?v=LMWMX43AvEA&list=PLYYZsFVm6afjy2NpPA2UYsiOjUOkNEwtm')
    elif album == 'mambole':
        browser.open('https://www.youtube.com/watch?v=YKfp_-ywEpA')   
    elif album == 'jose_augusto':
        browser.open('https://www.youtube.com/watch?v=jvJy7VieqeI')

def prevision_tiempo(tiempo=False, minmax=False):
    site = get('http://api.openweathermap.org/data/2.5/weather?q=arica&appid=a1f0d2c3fb0cbfb7d3230d60967b95a2&units=metric&lang=es')
    clima = site.json()
    #print(json.dumps(clima, indent=4))
    temperatura=clima['main']['temp']
    minima=clima['main']['temp_min']
    maxima=clima['main']['temp_max']
    descripcion=clima['weather'][0]['description']
    ciudad=clima['name']
    
    if tiempo:
        mensagem = f'Temperatura actual {temperatura} grados con {descripcion} en {ciudad}'
    if minmax:
        mensagem = f'Estado del tiempo, temperatura mínima de {minima} y máxima de {maxima} grados en {ciudad}'
    crear_audio(mensagem)

  
def main():
    while True:
        monitora_audio()

main()



Esperando el  comando: 
COMANDO:  atenea tocar mambole 
Esperando el  comando: 
Google no entiende ese audio
Esperando el  comando: 
Could not request results from Google Cloud Speech service; <HttpError 400 when requesting https://speech.googleapis.com/v1/speech:recognize?alt=json returned "Sync input too long. For audio longer than 1 min use LongRunningRecognize with a 'uri' parameter.">
Esperando el  comando: 
Could not request results from Google Cloud Speech service; <HttpError 400 when requesting https://speech.googleapis.com/v1/speech:recognize?alt=json returned "Sync input too long. For audio longer than 1 min use LongRunningRecognize with a 'uri' parameter.">
Esperando el  comando: 
Could not request results from Google Cloud Speech service; <HttpError 400 when requesting https://speech.googleapis.com/v1/speech:recognize?alt=json returned "Sync input too long. For audio longer than 1 min use LongRunningRecognize with a 'uri' parameter.">
Esperando el  comando: 
Esperando el  c

timeout: _ssl.c:1106: The handshake operation timed out